#### Notebook for running Chain-of-Thought with no supporting context experiments

In [1]:
import sys, os
sys.path.append('..')
root = '../root/'

In [2]:
from util import summarize_trial, log_trial, save_agents
import joblib
from agents import CoTAgent, ReflexionStrategy

#### Load the Sample

In [3]:
hotpot = joblib.load('../data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

In [4]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files='D:/files_new/10-samples.json', split='train')

In [10]:
row = train_dataset[1]

#### Define the Reflexion Strategy

In [6]:
print(ReflexionStrategy.__doc__)


    NONE: No reflection
    LAST_ATTEMPT: Use last reasoning trace in context 
    REFLEXION: Apply reflexion to the next reasoning trace 
    LAST_ATTEMPT_AND_REFLEXION: Use last reasoning trace in context and apply reflexion to the next reasoning trace 
    


In [7]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

In [8]:
from prompts import cot_simple_reflect_agent_prompt, cot_simple_reflect_prompt, cot_simple_agent_prompt
from fewshots import COTQA_SIMPLE6, COT_SIMPLE_REFLECTION

In [11]:
agent = CoTAgent(question=row['question'],
                   context='',
                   key=row['answer'],
                   agent_prompt=cot_simple_agent_prompt if strategy == ReflexionStrategy.NONE else cot_simple_reflect_agent_prompt,
                   cot_examples=COTQA_SIMPLE6,
                   reflect_prompt=cot_simple_reflect_prompt,
                   reflect_examples=COT_SIMPLE_REFLECTION)

In [17]:
agent

In [18]:
agent.run(reflexion_strategy = strategy)
#print(f'Answer: {agent.key}')

Running Reflexion strategy...
You have attempted to answer following question before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly answering the given question.
Reflections:
- My reasoning for the reversal of the given sentence in Arabic failed because I misunderstood the correct word order for the reversed sentence. In the future, when attempting similar questions, I should focus on practicing more with sentence structure and word order in Arabic to ensure accuracy. To mitigate this failure, I will review Arabic grammar rules related to sentence structure and practice more sentence reversal exercises to improve my understanding.
- My reasoning for reversing the given Arabic sentence failed because I did not consider the correct word order in Arabic. In the future, when attempting this type of question, I should focus on understanding the grammar rules of Ara

#### Initialize a CoTAgent for each question

In [19]:
from prompts import cot_simple_reflect_agent_prompt, cot_simple_reflect_prompt, cot_simple_agent_prompt
from fewshots import COTQA_SIMPLE6, COT_SIMPLE_REFLECTION

# Ensure `strategy` and `train_dataset` are properly defined elsewhere in your code
# Ensure `CoTAgent` and `ReflexionStrategy` are properly imported or defined elsewhere in your code

agents = [CoTAgent(question=row['question'],
                   context='',
                   key=row['answer'],
                   agent_prompt=cot_simple_agent_prompt if strategy == ReflexionStrategy.NONE else cot_simple_reflect_agent_prompt,
                   cot_examples=COTQA_SIMPLE6,
                   reflect_prompt=cot_simple_reflect_prompt,
                   reflect_examples=COT_SIMPLE_REFLECTION)
          for row in train_dataset]  # Directly iterate over `train_dataset` if it's a list of dictionaries


In [16]:
n = 2
trial = 0
agent.run(reflexion_strategy = strategy)
print(f'Answer: {agent.key}')
trial += 1
#log += log_trial(agent, trial)
correct, incorrect = summarize_trial(agent)
print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

Running Reflexion strategy...
You have attempted to answer following question before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly answering the given question.
Reflections:
- My reasoning for the reversal of the given sentence in Arabic failed because I misunderstood the correct word order for the reversed sentence. In the future, when attempting similar questions, I should focus on practicing more with sentence structure and word order in Arabic to ensure accuracy. To mitigate this failure, I will review Arabic grammar rules related to sentence structure and practice more sentence reversal exercises to improve my understanding.
- My reasoning for reversing the given Arabic sentence failed because I did not consider the correct word order in Arabic. In the future, when attempting this type of question, I should focus on understanding the grammar rules of Ara

TypeError: 'CoTAgent' object is not iterable

#### Run `n` trials

In [20]:
n = 5
trial = 0
log = ''
for i in range(n):
    for agent in [a for a in agents if not a.is_correct()]:
        agent.run(reflexion_strategy = strategy)
        print(f'Answer: {agent.key}')
    trial += 1
    log += log_trial(agents, trial)
    correct, incorrect = summarize_trial(agents)
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

Thought: لنفكر خطوة بخطوة. الجملة الأصلية هي "إنني أتطلع إلى نهاية هذا الأسبوع." يمكننا صياغتها بشكل مختلف عن طريق قول "أنا متحمس لنهاية هذا الأسبوع."
Action: Finish[أنا متحمس لنهاية هذا الأسبوع]
Answer: "لا أطيق صبرا حتى نهاية هذا الأسبوع."
Thought: دعونا نفكر خطوة بخطوة. سنختار عنصرًا عشوائيًا من القائمة.
Action: Finish[برتقال]
Answer: أحد العناصر العشوائية من القائمة [التفاح، البرتقال، الكمثرى، الفراولة] هو:
البرتقال.
Thought: دعنا نفكر خطوة بخطوة. الجملة الأمرية للفعل "أسق" هي "اسق النباتات".
Action: Finish[اسق النباتات]
Answer: تأكد من سقي النباتات بانتظام للحفاظ على صحتها وحيويتها.
Thought: دعونا نفكر خطوة بخطوة. الجملة الإيجابية ستكون: يجب علي أن أكون دقيقًا ولا أرتكب أي أخطاء.
Action: Finish[يجب علي أن أكون دقيقًا ولا أرتكب أي أخطاء]
Answer: يشجعني على السعي لتحقيق الكمال والتميز.
Thought: دعنا نفكر خطوة بخطوة. الأشياء المستحيل تناولها مع وجبة الإفطار هي الأسماك الطازجة واللحوم الحمراء الثقيلة.
Action: Finish[الأسماك الطازجة واللحوم الحمراء الثقيلة]
Answer: وجبتي الغداء والعشاء

KeyboardInterrupt: 

#### Save the result log

In [27]:
with open(os.path.join(root, 'CoT', 'no_context', strategy.value, f'{len(agents)}_questions_{trial}_trials.txt'), 'w') as f:
    f.write(log)
save_agents(agents, os.path.join(root, 'CoT', 'no_context', strategy.value, 'agents'))